In [1]:
## packages' import
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse.linalg import eigs

## WARNING

Theory requires the assumption of domain separability in most of the practical cases. 

## 1 : Penalty method for inverse problems

$Q$ and $U$ are euclidean balls

### non-separable components:  $f(x) = min_{u \in U}\, F(x) + \mathcal{F}(u) + \frac{\lambda}{2}||T_x\,x + T_u\,u- t||^2$

$t$ is $d_x$ dimensional

$T_x \in \mathbb{R}^{d_x\times d_x}$

$T_u \in \mathbb{R}^{d_x \times d_u}$ might be $\mathbf{0}$

$F \in \mathcal{S}^{1,1}_{\mu_x,L_x}(Q)$

$\mathcal{F} \in \mathcal{S}^{1,1}_{\mu_u,L_u}(U)$

case (1): bad aux, small dimension

case (2): good aux, big dimension

In [3]:
## dimensional set-up
pre_prop = 0.5
pre_d = 10
dx = int(np.ceil(pre_prop*pre_d)+2)
du = int(pre_d-dx+2)
d = dx+du

## size of domains 
advantage = 1 # neutral at 1
neutral = 1 # neutral at 1
H_Q = neutral*dx*advantage
H_U = neutral*du
H_base = np.sqrt(H_Q**2 + H_U**2)

In [6]:
## basic functions

# construction of T_x (invertible required)
pre_pre_T_x = np.random.uniform(0,0.5,(dx,dx))
pre_T_x = pre_pre_T_x.T@pre_pre_T_x
T_x = pre_T_x + 0.001*np.eye(dx)
biggest_x = np.real(eigs(T_x.T@T_x,1)[0][0])
smallest_x = np.real(eigs(T_x.T@T_x,dx)[0][dx-1]) # at comment if dx too big and inherent lines below

# construction of T_u
T_u = np.random.uniform(0,1,(dx,du))
biggest_u = np.real(eigs(T_u.T@T_u,1)[0][0])
smallest_u = np.real(eigs(T_u.T@T_u,du)[0][du-1])

# construction of T
T = np.block([[T_x.T@T_x,T_x.T@T_u],[T_u.T@T_x,T_u.T@T_u]])

# factice choice of minimizer
t = np.random.normal(0,1,dx)
u_opt = np.random.uniform(-5,5,du)
x_opt = np.linalg.solve(T_x,t-T_u@u_opt)
x_opt_no_Tu = np.linalg.solve(T_x,t)

# biggest eigenvalue of sym' matrix, computed efficiently 
biggest = np.real(eigs(T,1)[0][0])
smallest = np.real(eigs(T,d)[0][d-1])

# lambda param for regularization (lambda should grow until a very large value but)
lambda_param = 5/biggest

# construction of F
mu_F = 0.005
L_F = 1
mu_x_practice = mu_F
L_x_practice = L_F + lambda_param*biggest_x

# construction of F_bis
mu_F_bis = 0.005
L_F_bis = 1
mu_u_practice = mu_F_bis
L_u_practice = L_F_bis + lambda_param*biggest_u

L_base_F_F_bis = np.max([L_F,L_F_bis])
L_base_practice = L_base_F_F_bis+lambda_param*biggest
mu_base_F_F_bis = np.min([mu_F,mu_F_bis])
mu_base_practice = mu_base_F_F_bis
mu_base_practice_advantageous = mu_base_F_F_bis+lambda_param*smallest

$d_p = 1$ 

In [7]:
## oracle cost model (FGD used to solve inner problem)
oracle_K,oracle_beta,oracle_r = dx,du*H_U*np.sqrt(2*L_u_practice),1/2
oracle_cost = lambda eta: oracle_K + oracle_beta/eta**oracle_r
oracle_np_K,oracle_np_beta,oracle_np_r = d,0,1
oracle_np_cost = lambda eta: oracle_np_K + oracle_np_beta/eta**oracle_np_r

## delta cost model
delta_h,delta_s = 1/2 * np.sqrt(8*H_Q**2*L_u_practice), 1/2
delta_cost = lambda eta: delta_h/delta_s * eta**delta_s
delta_np_h,delta_np_s = 0,1
delta_np_cost = lambda eta: delta_np_h/delta_np_s * eta**delta_np_s

In [20]:
t = [2,1,4]
t[0:3]

[2, 1, 4]

In [21]:
## solve (I)

N = 500

def solve_I_instance1(epsilon_target,N,kappa,L,R,beta,r,h,s,la,lb,mode='GD'):

    if mode=='FGD':
        if kappa>0:
            v = 1/(1+np.sqrt(kappa))
            convergence_term = L*R**2/2 * v**(N-1)
            q = v**(N-1-np.arange(N))
        else:
            convergence_term = 2*L*R**2 / N**2 # approx
            q = (np.arange(N)+1)**2 / N**2
    else:
        if kappa>0:
            v = (1-kappa)
            convergence_term = np.min([L*R**2/2 * v**(N) / ((1-v**(N))/kappa), L*R**2/2 / N])
            q = ((1-v**(N))/kappa)*v**(N-1-np.arange(N))
        
    Gamma_N_eps = epsilon_target - convergence_term
    
    delta_cost = lambda eta: h/s * eta**s
    Phi = lambda Na,Nb: np.sum(delta_cost(lb)*q[:Nb]) + np.sum(delta_cost(la)*q[N+1-Na:]) 
    Psi = lambda Na,Nb: np.sum(h/s*(r*beta/h)**(s/(r+s))*q[Nb:N-Na]**(r/(r+s)))
    bar_lambda = ((Gamma_N_eps - Phi(Na,Nb))/Psi(Na,Nb))**(-(r+s)/s)

In [ ]:
## solve (II)

N = 500

budget_limit = 0

### greedy for complex sets

## 2 : Stackelberg in game theory 

$Q$ will be the simplex set whereas $U$ will denote a norm-ball

## 3 : Progressive gradient in large scale deterministic optimization

no restriction